## Load drive and data

## Import lib


[]


In [ ]:

!python -m pip install --upgrade pip
!pip install numpy
!pip install tensorflow
!pip install matplotlib
!pip install pyngrok
!pip install opencv-python
!pip install torch==2.2.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html
import os
import numpy as np
from tensorflow import keras
from keras import layers
from tensorflow import data as tf_data
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
import cv2
from tensorflow.keras.optimizers import Adam


Looking in links: https://download.pytorch.org/whl/torch_stable.html
Checking PyTorch GPU availability...
PyTorch: GPU not available, using CPU.

Checking TensorFlow GPU availability...
TensorFlow: GPU not available, using CPU.



## Filter out corrupted images

## **Tạo và tải dataset cho mô hình phân loại ảnh**

### Giải thích ngắn gọn:
Đoạn mã trên thực hiện các bước sau:
1. **Xác định đường dẫn dataset**: `dataset_path` lưu đường dẫn đến thư mục chứa dữ liệu về các loại chuối.
2. **Thiết lập kích thước ảnh và batch size**: Các ảnh đầu vào được thay đổi kích thước về `(180, 180)` và chia thành các batch 16 ảnh.
3. **Tạo các tập dữ liệu**:
   - **Tập huấn luyện (`train_ds`)**: Được tạo từ thư mục `/train`, chứa ảnh dùng để huấn luyện mô hình.
   - **Tập kiểm tra (`val_ds`)**: Được tạo từ thư mục `/valid`, dùng để đánh giá hiệu quả mô hình trong quá trình huấn luyện.
   - **Tập kiểm thử (`test_ds`)**: Được tạo từ thư mục `/test`, dùng để kiểm tra mô hình sau khi huấn luyện.
4. **In ra thông tin về số lượng batch trong các tập**: Mã in ra số lượng batch trong mỗi tập dữ liệu huấn luyện, kiểm thử và kiểm tra.



In [25]:
# Đường dẫn đến thư mục dataset
dataset_path = "E:/banana_classification"

# Kích thước ảnh và batch size
image_size = (224, 224)
batch_size = 32

# Tạo tập huấn luyện và tập kiểm tra từ thư mục
train_ds = keras.utils.image_dataset_from_directory(
    f"{dataset_path}/train",  # Thư mục chứa dữ liệu huấn luyện
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical"  # Hoặc "int" nếu bạn muốn nhãn dưới dạng số nguyên
)

val_ds = keras.utils.image_dataset_from_directory(
    f"{dataset_path}/valid",  # Thư mục chứa dữ liệu kiểm tra
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical"
)

test_ds = keras.utils.image_dataset_from_directory(
    f"{dataset_path}/test",  # Thư mục chứa dữ liệu test
    image_size=image_size,
    batch_size=batch_size,
    label_mode="categorical"
)

# Kiểm tra cấu trúc dataset
print(f"Train dataset size: {len(train_ds)} batches")
print(f"Validation dataset size: {len(val_ds)} batches")
print(f"Test dataset size: {len(test_ds)} batches")
# In một batch dữ liệu từ train_ds và val_ds
for images, labels in train_ds.take(1):  # Lấy một batch từ train_ds
    print(f"Train Batch - Image Shape: {images.shape}, Labels: {labels.shape}")

for images, labels in val_ds.take(1):  # Lấy một batch từ val_ds
    print(f"Validation Batch - Image Shape: {images.shape}, Labels: {labels.shape}")


Found 11793 files belonging to 4 classes.
Found 1123 files belonging to 4 classes.
Found 562 files belonging to 4 classes.
Train dataset size: 369 batches
Validation dataset size: 36 batches
Test dataset size: 18 batches
Train Batch - Image Shape: (32, 224, 224, 3), Labels: (32, 4)
Validation Batch - Image Shape: (32, 224, 224, 3), Labels: (32, 4)


##  Visualize the data
Chi Don gian la hien ra 9 image de check xem data co loi k
==> Khong anh huong den viec training



4 classes : ['overripe'->0, 'ripe'->1, 'rotten'->2, 'unripe'->3]

### Giải thích label
**`label_mode="categorical"`** trong `keras.utils.image_dataset_from_directory` chỉ định cách thức mã hóa nhãn (labels) của các ảnh trong dataset. Cụ thể, khi sử dụng **`label_mode="categorical"`**, nhãn của mỗi ảnh sẽ được mã hóa dưới dạng **one-hot encoding**.

### Giải thích về One-Hot Encoding:

One-hot encoding là một cách mã hóa nhãn trong các bài toán phân loại nhiều lớp. Đối với mỗi lớp trong dataset, ta sẽ tạo một vector trong đó chỉ có một phần tử có giá trị là 1, còn lại tất cả các phần tử khác có giá trị là 0.

### Ví dụ:
Giả sử bạn có một dataset với 3 lớp: `ripe`, `unripe`, và `green`. Khi sử dụng `label_mode="categorical"`, mỗi nhãn sẽ được mã hóa như sau:

- **Lớp `ripe`** sẽ được mã hóa thành `[1, 0, 0]`
- **Lớp `unripe`** sẽ được mã hóa thành `[0, 1, 0]`
- **Lớp `green`** sẽ được mã hóa thành `[0, 0, 1]`

### Cách thức hoạt động:
1. **Dataset có 3 lớp**: Nếu bạn có 3 lớp (ví dụ, `ripe`, `unripe`, `green`), thì mô hình sẽ phân loại mỗi ảnh vào một trong ba lớp này.
2. **One-hot Encoding**: Nhãn của mỗi ảnh sẽ được biểu diễn dưới dạng một vector 3 chiều, trong đó chỉ số lớp của ảnh là `1` và tất cả các chỉ số lớp còn lại là `0`.

### Ví dụ cụ thể:
Giả sử bạn có một dataset với các thư mục sau:
```
/train
    /ripe
        banana1.jpg
        banana2.jpg
    /unripe
        banana3.jpg
        banana4.jpg
    /green
        banana5.jpg
        banana6.jpg
```
Khi gọi hàm:
```python
train_ds = keras.utils.image_dataset_from_directory(
    '/train',
    label_mode='categorical'
)
```
- Ảnh trong thư mục `ripe` sẽ được gán nhãn `[1, 0, 0]` (vì `ripe` là lớp đầu tiên).
- Ảnh trong thư mục `unripe` sẽ được gán nhãn `[0, 1, 0]`.
- Ảnh trong thư mục `green` sẽ được gán nhãn `[0, 0, 1]`.

### Tại sao sử dụng One-Hot Encoding:
- **Phân loại đa lớp**: One-hot encoding giúp mô hình học cách phân biệt từng lớp dựa trên sự khác biệt giữa các lớp.
- **Dễ dàng với hàm mất mát**: Khi sử dụng một số hàm mất mát (loss function) như **categorical crossentropy**, one-hot encoding giúp tính toán độ lỗi giữa nhãn dự đoán và nhãn thật dễ dàng hơn.

### Tóm lại:
Khi bạn sử dụng `label_mode="categorical"`, các nhãn của dataset sẽ được mã hóa dưới dạng one-hot encoding, giúp mô hình có thể phân loại chính xác các ảnh vào các lớp khác nhau trong bài toán phân loại đa lớp.

In [ ]:


# Hiển thị 9 ảnh từ dataset
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):  # Lấy 1 batch đầu tiên
    for i in range(9):  # Lặp qua 9 ảnh đầu tiên
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))  # Chuyển đổi ảnh về kiểu uint8 để hiển thị
        plt.title(np.argmax(labels[i]))  # Lấy chỉ số lớp từ one-hot vector
        plt.axis("off")
# In tên của các lớp
print(train_ds.class_names)


NameError: name 'train_ds' is not defined

<Figure size 1000x1000 with 0 Axes>

## Using image data augmentation
==> da dang hoa DATASET

In [ ]:
# data_augmentation_layers = [
#     layers.RandomFlip("horizontal"),
#     layers.RandomRotation(0.1),
# ]

# def data_augmentation(images):
#     for layer in data_augmentation_layers:
#         images = layer(images)
#     return images

### Hiển thị các ảnh sau khi bổ sung dữ liệu:
==> Kết quả cho thấy các biến thể khác nhau của cùng một ảnh, giúp bạn thấy rõ cách mà bổ sung dữ liệu có thể giúp mô hình học được nhiều đặc trưng tổng quát hơn, từ đó cải thiện khả năng xử lý ảnh thực tế.


In [ ]:
# plt.figure(figsize=(10, 10))
# for images, _ in train_ds.take(1):
#     for i in range(9):
#         augmented_images = data_augmentation(images)
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(np.array(augmented_images[0]).astype("uint8"))
#         plt.axis("off")

# Preprocessing
## Standardizing the data
==> Two options to preprocess the data:
1. Make it part of the model
2. apply it to the dataset ==> ko bt chon gi chon 2
=> This second option (asynchronous preprocessing) is always a solid choice.

In [26]:
# Preprocessing function for rescaling images
def preprocess_image(image, label):
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Standardize pixel values to the range [0, 1]
    return image, label

# Apply preprocessing to the datasets
train_ds = train_ds.map(preprocess_image)
val_ds = val_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

## BUILD MODEL
We'll build a small version of the Xception network. We haven't particularly tried to optimize the architecture;
if you want to do a systematic search for the best model configuration, consider using KerasTuner.

In [27]:
def make_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.25)(x)  # Thêm Dropout sau Activation trong Entry block

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.25)(x)  # Thêm Dropout sau BatchNormalization

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.25)(x)  # Thêm Dropout sau BatchNormalization

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.25)(x)  # Thêm Dropout sau Activation trong cuối các layers

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.5)(x)  # Thêm Dropout trước lớp Dense đầu ra

    # Output layer with 4 classes
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return tf.keras.Model(inputs, outputs)

# Tạo mô hình với num_classes = 4
model = make_model(input_shape=(224, 224, 3), num_classes=4)

# Tóm tắt mô hình
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 112, 112,  │      3,584 │ rescaling_1[0][0] │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 112, 112,  │          0 │ activation_8[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 112, 112,  │          0 │ dropout_9[0][0]   │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_7  │ (None, 112, 112,  │     34,176 │ activation_9[0][… │
│ (SeparableConv2D)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ separable_conv2d… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Dropout)           │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 112, 112,  │          0 │ dropout_10[0][0]  │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_8  │ (None, 112, 112,  │     68,096 │ activation_10[0]… │
│ (SeparableConv2D)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ separable_conv2d… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Dropout)           │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 56, 56,    │          0 │ dropout_11[0][0]  │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 56, 56,    │     33,024 │ dropout_9[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 56, 56,    │          0 │ max_pooling2d_3[

 Total params: 2,734,140 (10.43 MB)

 Trainable params: 2,725,852 (10.40 MB)

 Non-trainable params: 8,288 (32.38 KB)

## TRAIN


### Train_ds and val_ds
Tập `val_ds` (validation dataset) được sử dụng trong quá trình huấn luyện mô hình để theo dõi hiệu suất của mô hình trên dữ liệu chưa được huấn luyện, giúp đánh giá khả năng tổng quát của mô hình và tránh overfitting.

Trong mã huấn luyện, bạn đã sử dụng `validation_data=val_ds` để mô hình tính toán độ chính xác và mất mát trên tập xác thực sau mỗi epoch.

### Tóm tắt cách sử dụng:
- **`train_ds`**: Dữ liệu huấn luyện.
- **`val_ds`**: Dữ liệu xác thực, được sử dụng để kiểm tra mô hình trong quá trình huấn luyện.
- Mô hình được huấn luyện qua nhiều epoch, và hiệu suất của mô hình trên tập xác thực sẽ được theo dõi để đảm bảo không bị overfit.

Sau khi huấn luyện, bạn có thể truy cập kết quả từ `history` để kiểm tra độ chính xác và mất mát trên cả tập huấn luyện và tập xác thực.

In [28]:


# Biên dịch mô hình
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,                   # Tối ưu hóa với Adam
    loss='categorical_crossentropy',     # Hàm mất mát cho phân loại đa lớp
    metrics=['accuracy']
)

# Huấn luyện mô hình
history = model.fit(
    train_ds,
    # Dữ liệu huấn luyện
    validation_data=val_ds,  # Dữ liệu xác thực
    epochs=5,  # Số epoch (lặp qua toàn bộ dataset nhiều lần)
    batch_size=batch_size,  # Kích thước batch

)

# Kiểm tra độ chính xác của mô hình trên dữ liệu kiểm tra (test)
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc * 100:.2f}%")

# Lưu mô hình nếu cần
model.save("banana_ripeness_model.h5")


Epoch 1/5
369/369 ━━━━━━━━━━━━━━━━━━━━ 2596s 7s/step - accuracy: 0.7304 - loss: 0.6905 - val_accuracy: 0.3455 - val_loss: 1.5847
Epoch 2/5
 93/369 ━━━━━━━━━━━━━━━━━━━━ 32:20 7s/step - accuracy: 0.8659 - loss: 0.3778

KeyboardInterrupt: 

## Test model

In [ ]:


# Vẽ đồ thị loss và accuracy
# Đồ thị mất mát (loss)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss (train)')
plt.plot(history.history['val_loss'], label='Loss (validation)')
plt.title('Mất mát (Loss)')
plt.xlabel('Epoch')
plt.ylabel('Mất mát')
plt.legend()

# Đồ thị độ chính xác (accuracy)
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy (train)')
plt.plot(history.history['val_accuracy'], label='Accuracy (validation)')
plt.title('Độ chính xác (Accuracy)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Hiển thị đồ thị
plt.tight_layout()
plt.show()

test_loss, test_acc = model.evaluate(test_ds)

# In kết quả
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")


## Lấy trọng số tối ưu và thay vào C++ ==> chưa thực hiện được

In [ ]:
import tensorflow as tf

# Tải mô hình đã huấn luyện
model = tf.keras.models.load_model('save_at_10.keras')

# Chuyển sang TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Lưu mô hình TFLite
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


ValueError: File not found: filepath=save_at_10.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


In [ ]:
import tensorflow as tf

# Tải mô hình đã lưu
model = tf.keras.models.load_model("/content/save_at_10.keras")

# Chuyển sang định dạng TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("/content/save_at_10.keras")
tflite_model = converter.convert()

# Lưu tệp .tflite
with open("model.tflite", "wb") as f:
    f.write(tflite_model)
